In [1]:
# PROJECT : Purchasing Price Optimization
# RESEARCHER : Hyun Ho Park
# Created At : 2023-03-07
# Updated At : 2023-03-07

In [221]:
'''
Import library
'''

import pandas as pd
import numpy as np
from datetime import datetime

import os

from tqdm import tqdm

import matplotlib.pyplot as plt

import gc

print(f'DONE.')

DONE.


In [3]:
'''
IMPORT AUTOML Library
'''

# solve memory issue
#!pip install --upgrade pycaret

from pycaret.regression import *

print(f'DONE')

DONE


In [4]:
'''
Version Check
'''

print(f'Pandas version : {pd.__version__}.')

Pandas version : 1.4.2.


In [164]:
'''
Function
'''

# Subtract n month
def subtract_n_month(data, n_month):

    # one_month
    if n_month == 1 :
        if data.month == 1:
            # If the current month is January, subtract one year and set the month to December
            n_month_ago = data.replace(year=data.year-1, month=12)
        else:
            # Otherwise, just subtract one month
            n_month_ago = data.replace(month=data.month-1)

    # two month
    elif n_month == 2:
        if data.month == 1:
            # If the current month is January, subtract one year and set the month to November
            n_month_ago = data.replace(year=data.year-1, month=11)
        elif data.month == 2:
            # If the current month is February, subtract one year and set the month to December
            n_month_ago = data.replace(year=data.year-1, month=12)
        else:
            # Otherwise, just subtract one month
            n_month_ago = data.replace(month=data.month-2)

    # three month
    elif n_month == 3:
        if data.month == 1:
            # If the current month is January, subtract one year and set the month to November
            n_month_ago = data.replace(year=data.year-1, month=10)
        elif data.month == 2:
            # If the current month is February, subtract one year and set the month to December
            n_month_ago = data.replace(year=data.year-1, month=11)
        elif data.month == 3:
            # If the current month is February, subtract one year and set the month to December
            n_month_ago = data.replace(year=data.year-1, month=12)
        else:
            # Otherwise, just subtract one month
            n_month_ago = data.replace(month=data.month-3)
    else:
        return "NEED TO INPUT n_month from 1 to 3"

    return datetime.strftime(n_month_ago, '%Y-%m')

In [47]:
'''
Load Dataset

- In production level, datasets are cleansed
a'''

normalized_df = pd.read_excel('../1. Frozen shrimp imported china/1. shrimp_research.xlsx', sheet_name='btd_post_process')
print(f'DONE.')

DONE.


In [48]:
# Data Check

print(f'Data size : {len(normalized_df)}')
normalized_df.head()

Data size : 69493


,Flow,HS Code,HS_Product,Date,Importer,Country of Importer,Country of Exporter,Exporter,Unit_Qty,Quantity,...,Value_Local_Currency,Incoterms,Transport,DataId,re_cal_usd_per_kg,re_cal_usd_per_qty,zero_data,rename_importer,rename_exporter,INFO
0,Export,306179990,Los demás,2022-05-09,CHINA SDIC INTERNACIONAL TRADE CO LTD,China (CN),Ecuador (EC),FRIGOPESCA C.A.,KILOGRAMO GRUTO,38480.0,...,0.0,NaN,Water,%252fFAPHYNOS%252fLuhkmINku5Dw%253d%253d,19.230769,12.993763,OK,"CHINA SDCI INTERNACIONAL TRADE CO., LTD",FRIGOPESCA C.A.,동일한 날짜 가격/중량 통일
1,Export,306179990,Los demás,2022-05-09,CHINA SDIC INTERNACIONAL TRADE CO LTD,China (CN),Ecuador (EC),FRIGOPESCA C.A.,KILOGRAMO GRUTO,38480.0,...,0.0,NaN,Water,lEyWX9g5SgUTDrxEVHWzJQ%253d%253d,19.230769,12.993763,OK,"CHINA SDCI INTERNACIONAL TRADE CO., LTD",FRIGOPESCA C.A.,동일한 날짜 가격/중량 통일
2,Export,306179990,Los demás,2022-05-09,CHINA SDIC INTERNACIONAL TRADE CO LTD,China (CN),Ecuador (EC),FRIGOPESCA C.A.,KILOGRAMO GRUTO,38480.0,...,0.0,NaN,Water,qoxQkpCrpmNjOIAgsQKklA%253d%253d,19.230769,12.993763,OK,"CHINA SDCI INTERNACIONAL TRADE CO., LTD",FRIGOPESCA C.A.,동일한 날짜 가격/중량 통일
3,Export,306179990,Los demás,2022-05-09,CHINA SDIC INTERNACIONAL TRADE CO LTD,China (CN),Ecuador (EC),FRIGOPESCA C.A.,KILOGRAMO GRUTO,38480.0,...,0.0,NaN,Water,qoxQkpCrpmObp2r5cURJYw%253d%253d,19.230769,12.993763,OK,"CHINA SDCI INTERNACIONAL TRADE CO., LTD",FRIGOPESCA C.A.,동일한 날짜 가격/중량 통일
4,Export,306179990,Los demás,2022-05-09,"QINGDAO KAITOU INTERNATIONAL TRADING CO.,LTD",China (CN),Ecuador (EC),FRIGOPESCA C.A.,KILOGRAMO GRUTO,38480.0,...,0.0,NaN,Water,qoxQkpCrpmM2RHbKa%252fysVg%253d%253d,19.230769,12.993763,OK,QINGDAO KAITOU INTERNATIONAL,FRIGOPESCA C.A.,동일한 날짜 가격/중량 통일


### Preprocessing

In [240]:
'''
Select only data is existed
'''

data_df = normalized_df[normalized_df['zero_data'] == 'OK'].copy()
data_df['month'] = data_df['Date'].apply(lambda x: datetime.strftime(x, '%Y-%m'))

print(f'LENGTH : {len(data_df)}')
data_df.head()

LENGTH : 56015


,Flow,HS Code,HS_Product,Date,Importer,Country of Importer,Country of Exporter,Exporter,Unit_Qty,Quantity,...,Incoterms,Transport,DataId,re_cal_usd_per_kg,re_cal_usd_per_qty,zero_data,rename_importer,rename_exporter,INFO,month
0,Export,306179990,Los demás,2022-05-09,CHINA SDIC INTERNACIONAL TRADE CO LTD,China (CN),Ecuador (EC),FRIGOPESCA C.A.,KILOGRAMO GRUTO,38480.0,...,NaN,Water,%252fFAPHYNOS%252fLuhkmINku5Dw%253d%253d,19.230769,12.993763,OK,"CHINA SDCI INTERNACIONAL TRADE CO., LTD",FRIGOPESCA C.A.,동일한 날짜 가격/중량 통일,2022-05
1,Export,306179990,Los demás,2022-05-09,CHINA SDIC INTERNACIONAL TRADE CO LTD,China (CN),Ecuador (EC),FRIGOPESCA C.A.,KILOGRAMO GRUTO,38480.0,...,NaN,Water,lEyWX9g5SgUTDrxEVHWzJQ%253d%253d,19.230769,12.993763,OK,"CHINA SDCI INTERNACIONAL TRADE CO., LTD",FRIGOPESCA C.A.,동일한 날짜 가격/중량 통일,2022-05
2,Export,306179990,Los demás,2022-05-09,CHINA SDIC INTERNACIONAL TRADE CO LTD,China (CN),Ecuador (EC),FRIGOPESCA C.A.,KILOGRAMO GRUTO,38480.0,...,NaN,Water,qoxQkpCrpmNjOIAgsQKklA%253d%253d,19.230769,12.993763,OK,"CHINA SDCI INTERNACIONAL TRADE CO., LTD",FRIGOPESCA C.A.,동일한 날짜 가격/중량 통일,2022-05
3,Export,306179990,Los demás,2022-05-09,CHINA SDIC INTERNACIONAL TRADE CO LTD,China (CN),Ecuador (EC),FRIGOPESCA C.A.,KILOGRAMO GRUTO,38480.0,...,NaN,Water,qoxQkpCrpmObp2r5cURJYw%253d%253d,19.230769,12.993763,OK,"CHINA SDCI INTERNACIONAL TRADE CO., LTD",FRIGOPESCA C.A.,동일한 날짜 가격/중량 통일,2022-05
4,Export,306179990,Los demás,2022-05-09,"QINGDAO KAITOU INTERNATIONAL TRADING CO.,LTD",China (CN),Ecuador (EC),FRIGOPESCA C.A.,KILOGRAMO GRUTO,38480.0,...,NaN,Water,qoxQkpCrpmM2RHbKa%252fysVg%253d%253d,19.230769,12.993763,OK,QINGDAO KAITOU INTERNATIONAL,FRIGOPESCA C.A.,동일한 날짜 가격/중량 통일,2022-05


In [241]:
gc.collect()

0

In [242]:
'''
Check there is abnormal in the Net_weight

For preprocessing, less than 1000 kg is hard to imagine
multiple 1000 (ton)
'''

data_df.loc[
    data_df[(data_df['Net_Weight'] < 500) & (data_df['Quantity'] > 5000)].index,
    'Net_Weight'
] *= 1000


In [288]:
'''
Group by the month, importer/exporter
'''

importer_df = data_df.groupby(by= [
    'rename_importer', 'month'
])['Quantity', 'Net_Weight', 'Value (USD)']

importer_df = importer_df.sum().reset_index()

importer_df[['M-1 Price', 'M-2 Price', 'M-3 Price']] = None

importer_df['HS code'] = '030617'

print(f'IMPORT DATA : {len(importer_df)}')

importer_df.rename(columns = {
    "Value (USD)" : "Target Price"
}, inplace=True)

importer_df.head()

IMPORT DATA : 1885


,rename_importer,month,Quantity,Net_Weight,Target Price,M-1 Price,M-2 Price,M-3 Price,HS code
0,& NOTIFY: SHANGHAI VANNAWIN TRADING,2022-07,17280.00,17280.0,112148.31,None,None,None,030617
1,ABAD ADUANAS Y LOGISTICA VALENCIASL,2022-08,25348.63,23260.0,102344.00,None,None,None,030617
2,"AHCOF INDUSTRIAL DEVELOPMENT CO., LTD.",2022-01,92400.00,72000.0,1050000.00,None,None,None,030617
3,"AHCOF INDUSTRIAL DEVELOPMENT CO., LTD.",2022-05,79960.00,66360.0,820520.13,None,None,None,030617
4,"AHCOF INDUSTRIAL DEVELOPMENT CO., LTD.",2022-06,170853.59,148480.0,1246864.48,None,None,None,030617


In [289]:
%%time

month_lst = np.arange(1, 4)

for i in tqdm(importer_df.index):
    target_company = importer_df.loc[i, 'rename_importer']

    for month_ in month_lst:
        # M-month price
        month_n_price = subtract_n_month(pd.to_datetime(importer_df.loc[i, 'month']), month_)
        month_n_data = importer_df[(importer_df['rename_importer'] == target_company) & (importer_df['month'] == month_n_price)]

        # Apply data
        column_ = str('M-' + str(month_) + ' Price')
        if month_n_data.empty:
            importer_df.loc[i, column_] = None
        else:
            importer_df.loc[i, column_] = month_n_data['Target Price'].iloc[0]

    del target_company, month_n_price, month_n_data, column_

print(f'ALL DONE.')

100%|██████████| 1885/1885 [00:04<00:00, 416.21it/s]

ALL DONE.
CPU times: user 4.48 s, sys: 138 ms, total: 4.62 s
Wall time: 4.53 s


In [290]:
# Return the result

importer_df.head(10)

,rename_importer,month,Quantity,Net_Weight,Target Price,M-1 Price,M-2 Price,M-3 Price,HS code
0,& NOTIFY: SHANGHAI VANNAWIN TRADING,2022-07,17280.00,17280.0,112148.31,None,None,None,030617
1,ABAD ADUANAS Y LOGISTICA VALENCIASL,2022-08,25348.63,23260.0,102344.00,None,None,None,030617
2,"AHCOF INDUSTRIAL DEVELOPMENT CO., LTD.",2022-01,92400.00,72000.0,1050000.00,None,None,None,030617
3,"AHCOF INDUSTRIAL DEVELOPMENT CO., LTD.",2022-05,79960.00,66360.0,820520.13,None,None,None,030617
4,"AHCOF INDUSTRIAL DEVELOPMENT CO., LTD.",2022-06,170853.59,148480.0,1246864.48,820520.13,None,None,030617
5,"AHCOF INDUSTRIAL DEVELOPMENT CO., LTD.",2022-07,630104.77,572920.0,5145946.98,1246864.48,820520.13,None,030617
6,"AHCOF INDUSTRIAL DEVELOPMENT CO., LTD.",2022-08,196553.59,188880.0,1378415.26,5145946.98,1246864.48,820520.13,030617
7,"AHCOF INDUSTRIAL DEVELOPMENT CO., LTD.",2022-09,75200.00,55600.0,281172.03,1378415.26,5145946.98,1246864.48,030617
8,"AHCOF INDUSTRIAL DEVELOPMENT CO., LTD.",2022-10,114240.00,114240.0,637957.76,281172.03,1378415.26,5145946.98,030617
9,ALIAN AQC PRODUCTS LTD,2022-08,35640.00,35640.0,218825.96,None,None,None,030617


In [291]:
# Get clean data

train_import = importer_df.dropna().sample(frac=1).reset_index(drop=True)

print(f'DATA SIZE : {len(train_import)}')
train_import.head()

DATA SIZE : 599


,rename_importer,month,Quantity,Net_Weight,Target Price,M-1 Price,M-2 Price,M-3 Price,HS code
0,"ZHANJIANG SEA WORLD INTERNATIONAL TRADE CO., LTD",2022-10,2576202.25,2025540.0,12356538.82,3610256.71,20615405.07,17428042.57,030617
1,CHINATEX RAW MATERIAL INTERNATIONAL TRADING CO...,2022-08,39420.00,39420.0,270563.82,935818.76,1554945.18,1093104.0,030617
2,ZHANJIANG SHENBAO DEVELOPMENT CO,2022-07,442640.00,442640.0,2938739.89,2328073.79,1245766.56,1166414.01,030617
3,TAIZHOU FEIFAN INTERNATIONAL,2022-05,102420.00,75120.0,1021327.12,600000.0,1729394.44,4193487.24,030617
4,ZHANJIANG YUEZHIDU FOOD CO LTD,2022-10,128520.00,128520.0,673586.60,1425793.88,1543018.08,105909.05,030617


### PYCARET

In [328]:
'''
Convert data for training
'''

# importer
#pycaret_importer = train_import[train_import.columns.drop(['month', 'Quantity', 'Net_Weight', 'HS code'])].copy()

pycaret_importer = train_import.copy()

print(f'DATA SIZE : {len(pycaret_importer)}\n')
pycaret_importer.head()

DATA SIZE : 599



,rename_importer,month,Quantity,Net_Weight,Target Price,M-1 Price,M-2 Price,M-3 Price,HS code
0,"ZHANJIANG SEA WORLD INTERNATIONAL TRADE CO., LTD",2022-10,2576202.25,2025540.0,12356538.82,3610256.71,20615405.07,17428042.57,030617
1,CHINATEX RAW MATERIAL INTERNATIONAL TRADING CO...,2022-08,39420.00,39420.0,270563.82,935818.76,1554945.18,1093104.0,030617
2,ZHANJIANG SHENBAO DEVELOPMENT CO,2022-07,442640.00,442640.0,2938739.89,2328073.79,1245766.56,1166414.01,030617
3,TAIZHOU FEIFAN INTERNATIONAL,2022-05,102420.00,75120.0,1021327.12,600000.0,1729394.44,4193487.24,030617
4,ZHANJIANG YUEZHIDU FOOD CO LTD,2022-10,128520.00,128520.0,673586.60,1425793.88,1543018.08,105909.05,030617


In [329]:
'''
Apply Automl
'''

set_up = setup(data = pycaret_importer, target='Target Price', n_jobs=10)

,Description,Value
0,Session id,6990
1,Target,Target Price
2,Target type,regression
3,Data shape,"(599, 9)"
4,Train data shape,"(419, 9)"
5,Test data shape,"(180, 9)"
6,Numeric features,2
7,Categorical features,6
8,Preprocess,True
9,Imputation type,simple


In [330]:
'''
Because of memory issue

apply models into differently

‘lr’ - Linear Regression
‘lasso’ - Lasso Regression
‘ridge’ - Ridge Regression
‘en’ - Elastic Net
‘lar’ - Least Angle Regression
‘llar’ - Lasso Least Angle Regression
‘omp’ - Orthogonal Matching Pursuit
‘br’ - Bayesian Ridge
‘ard’ - Automatic Relevance Determination
‘par’ - Passive Aggressive Regressor
‘ransac’ - Random Sample Consensus
‘tr’ - TheilSen Regressor
‘huber’ - Huber Regressor
‘kr’ - Kernel Ridge
‘svm’ - Support Vector Regression
‘knn’ - K Neighbors Regressor
‘dt’ - Decision Tree Regressor
‘rf’ - Random Forest Regressor
‘et’ - Extra Trees Regressor
‘ada’ - AdaBoost Regressor
‘gbr’ - Gradient Boosting Regressor
‘mlp’ - MLP Regressor
‘xgboost’ - Extreme Gradient Boosting
‘lightgbm’ - Light Gradient Boosting Machine
‘catboost’ - CatBoost Regressor
'''

print('LIST...')

LIST...


In [331]:
gc.collect()


941

In [340]:
# Train the first list

best_1 = compare_models(n_select=5,
                        include = [
                            'lr',
                            'lasso',
                            'ridge',
                            'en',
                            'lar',
                            'llar',
                            'omp',
                            'br',
                            'ard',
                            'par',
                            'ransac',
                            'tr',
                        ])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
tr,TheilSen Regressor,841431.9501,6780386770995.6006,2272866.7568,0.9425,0.3695,0.2692,0.1830
ransac,Random Sample Consensus,868373.3296,6855966364758.4404,2276673.5875,0.9418,0.4576,0.2806,0.0220
omp,Orthogonal Matching Pursuit,916287.8002,5949749217355.9658,2155274.2685,0.9407,0.7300,0.5651,0.0190
par,Passive Aggressive Regressor,1205380.2106,6509185218115.3027,2290457.0601,0.9365,0.8946,1.2059,0.0190
ard,Automatic Relevance Determination,977887.9063,6146684505469.9316,2216668.8823,0.9364,0.8627,0.6477,0.0200
lr,Linear Regression,994970.0639,6139748241440.7793,2220120.2266,0.9360,0.8660,0.6972,0.7270
lasso,Lasso Regression,994965.0339,6139733551849.7637,2220116.9688,0.9360,0.8657,0.6971,0.0220
ridge,Ridge Regression,994970.0639,6139748241440.7285,2220120.2266,0.9360,0.8660,0.6972,0.0230
en,Elastic Net,994965.0339,6139733551839.2441,2220116.9688,0.9360,0.8657,0.6971,0.0180
lar,Least Angle Regression,994970.0639,6139748241440.8223,2220120.2266,0.9360,0.8660,0.6972,0.0210


In [343]:
# Train the second list

best_2 = compare_models(n_select=5,
                        include = [
                            'huber',
                            'kr',
                            'svm',
                            'knn',
                            'dt',
                            'rf',
                            'et',
                            'ada',
                            'gbr',
                            'mlp',
                            'lightgbm',
                        ])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,842488.9483,6201885492545.2607,2176161.0548,0.9452,0.4680,0.2869,0.0370
mlp,MLP Regressor,966773.3957,5661150464702.7373,2159416.7477,0.9421,0.7449,0.6422,0.1330
kr,Kernel Ridge,987366.6949,6064983361250.3496,2191706.0874,0.9398,0.7858,0.6249,0.0230
et,Extra Trees Regressor,1005067.3656,11131370971343.8730,2762901.1265,0.9170,0.2873,0.2354,0.0500
gbr,Gradient Boosting Regressor,1047656.9696,11866647454556.7461,2873291.8339,0.9134,0.3933,0.3403,0.0320
rf,Random Forest Regressor,1011589.9495,11036280469040.4062,2782475.7292,0.9133,0.2880,0.2354,0.0550
ada,AdaBoost Regressor,1600895.5053,14299939795910.1445,3321420.5355,0.8604,1.1145,2.3152,0.0300
dt,Decision Tree Regressor,1330434.4383,19502965824655.0625,3694508.4921,0.8472,0.3697,0.3008,0.0280
knn,K Neighbors Regressor,2101932.3750,39721829190860.7969,5697855.2250,0.4217,0.8825,1.5892,0.0220
svm,Support Vector Regression,4156153.9418,120336716378962.0781,10052654.1643,-0.1319,1.5180,2.3106,0.0220


In [345]:
'''
Check the model performance
'''

print(f'BEST LIST : {best_1}\n')

print(f'BEST LIST : {best_2}\n')

BEST LIST : [TheilSenRegressor(max_iter=1000, n_jobs=10, random_state=6990), RANSACRegressor(random_state=6990), OrthogonalMatchingPursuit(), PassiveAggressiveRegressor(random_state=6990), ARDRegression(n_iter=1000)]

BEST LIST : [HuberRegressor(), MLPRegressor(max_iter=500, random_state=6990), KernelRidge(), ExtraTreesRegressor(n_jobs=10, random_state=6990), GradientBoostingRegressor(random_state=6990)]



In [349]:
# Predict model

model_ = best_2
rank_ = 0

print(f'MODEL NAME : {model_[rank_]}\n')
predict_model(model_[rank_], data = pycaret_importer)

MODEL NAME : HuberRegressor()



,rename_importer,month,Quantity,Net_Weight,M-1 Price,M-2 Price,M-3 Price,HS code,Label
0,"ZHANJIANG SEA WORLD INTERNATIONAL TRADE CO., LTD",2022-10,2576202.25,2025540.0,3610256.71,20615405.07,17428042.57,030617,1.820390e+07
1,CHINATEX RAW MATERIAL INTERNATIONAL TRADING CO...,2022-08,39420.00,39420.0,935818.76,1554945.18,1093104.0,030617,2.074181e+05
2,ZHANJIANG SHENBAO DEVELOPMENT CO,2022-07,442640.00,442640.0,2328073.79,1245766.56,1166414.01,030617,3.047241e+06
3,TAIZHOU FEIFAN INTERNATIONAL,2022-05,102420.00,75120.0,600000.0,1729394.44,4193487.24,030617,8.218584e+05
4,ZHANJIANG YUEZHIDU FOOD CO LTD,2022-10,128520.00,128520.0,1425793.88,1543018.08,105909.05,030617,8.147618e+05
...,...,...,...,...,...,...,...,...,...
594,HANGZHOU GEXIN IMPORT & EXPORT CO.,2022-05,815250.00,815250.0,3489893.58,2105480.23,3946382.88,030617,5.710861e+06
595,CHANGSHA COUNTRY PILOT INTERNATIONAL TRADE CO....,2022-06,840420.00,732972.0,5053863.88,11886968.72,8109694.36,030617,5.910600e+06
596,CONSIGNEE/THIRD PARTY :,2022-12,73440.00,73440.0,1008574.91,2551965.83,1227877.94,030617,3.532756e+05
597,"GUANGDONG HONGBAO AQUATIC DEVELOPM ENT CO., LTD.",2022-06,2702101.72,2469505.0,13875799.52,9113067.04,8901232.3,030617,1.903454e+07


In [350]:
pycaret_importer

,rename_importer,month,Quantity,Net_Weight,Target Price,M-1 Price,M-2 Price,M-3 Price,HS code
0,"ZHANJIANG SEA WORLD INTERNATIONAL TRADE CO., LTD",2022-10,2576202.25,2025540.0,12356538.82,3610256.71,20615405.07,17428042.57,030617
1,CHINATEX RAW MATERIAL INTERNATIONAL TRADING CO...,2022-08,39420.00,39420.0,270563.82,935818.76,1554945.18,1093104.0,030617
2,ZHANJIANG SHENBAO DEVELOPMENT CO,2022-07,442640.00,442640.0,2938739.89,2328073.79,1245766.56,1166414.01,030617
3,TAIZHOU FEIFAN INTERNATIONAL,2022-05,102420.00,75120.0,1021327.12,600000.0,1729394.44,4193487.24,030617
4,ZHANJIANG YUEZHIDU FOOD CO LTD,2022-10,128520.00,128520.0,673586.60,1425793.88,1543018.08,105909.05,030617
...,...,...,...,...,...,...,...,...,...
594,HANGZHOU GEXIN IMPORT & EXPORT CO.,2022-05,815250.00,815250.0,5887626.70,3489893.58,2105480.23,3946382.88,030617
595,CHANGSHA COUNTRY PILOT INTERNATIONAL TRADE CO....,2022-06,840420.00,732972.0,6002865.50,5053863.88,11886968.72,8109694.36,030617
596,CONSIGNEE/THIRD PARTY :,2022-12,73440.00,73440.0,362963.90,1008574.91,2551965.83,1227877.94,030617
597,"GUANGDONG HONGBAO AQUATIC DEVELOPM ENT CO., LTD.",2022-06,2702101.72,2469505.0,19355617.15,13875799.52,9113067.04,8901232.3,030617
